# RoBERTa Model Pre-Training

In this notebook we will train a transformer based model more specifically `KantaiBERT` model using a data set based on three books by Immanuel Kant. 

[Orignal Sourcecode](https://github.com/PacktPublishing/Transformers-for-Natural-Language-Processing/blob/main/Chapter03/KantaiBERT.ipynb)

We will first create the dataset, train a tokenizer(mehh basically word2tokenid, tokenid2word functionality) and then we will pretrain the model that can be used on downstream tasks.

The model we are trying to build is a `Robustly Optimized BERT Pretraining Approach (RoBERTa)` like model which is based on a BERT architecture.

The initial BERT models are udertrained compared to the data it was given. RoBERTa models were introduced to fix that issue by providing improved mechanisms to the pretraining process. Also it increase the training performance of downstreaming tasks as well. The techniques used by RoBERTa is interesting and it is worth reading!


### General Details:

In this notebook, KantaiBERT will be trained as a small model with 6 layers, 12 heads of attention. Also it will utilize a Byte-Level Byte-Pair encoding tokenizer like the GPT2 models. Also the sequences will be segmented using a separation token \</s> rather than using token type ids.

We will train the model with Masked Language Modelling technique. 


#### 1. Loading the dataset

We are using a dataset consist of 3 books data. It is available in the data folder of the project. Original data is from [Project Gutenberg](https://www.gutenberg.org).

#### 2. Training the tokenizer

We will use the HuggingFace provided ByteLevelBPETokenizer as the tokenizer for our model. This also will break the words in to substring/ subword. But the approach is bit different. 
[A medium article on BytePair Encoding](https://towardsdatascience.com/byte-pair-encoding-subword-based-tokenization-algorithm-77828a70bee0)


Below are some parameter values we will use for the tokenizer.

    - files: Path for the data

    - vocab_size: This is the required vocabulary size. We will use 52000.

    - min_frequency: This is the threshold for selecting a token. We will use 2.
    
    - special_tokens: The list of special tokens we need to have. In our case \<s>, \<pad>, \</s>, \<unk>, \<mask>.

In [1]:
from lib2to3.pgen2 import token
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files='data/kant.txt', vocab_size=52000, min_frequency=2, 
                special_tokens=[
                    "<s>",
                    "<pad>",
                    "</s>",
                    "<unk>",
                    "<mask>"
                ])

So now our tokenizer is trained. We can use it to our later works.
But is is beneficial to save the tokenizer to disk, so we can use it later any time. When we save the tokenizers, it usually creates 2 files.

1. merges.txt --> which contains merged tokenized sub-strings
2. vocab.json --> which contains the indices of the tokenized sub-strings.

(Apparently the weird 'Ġ' symbol denotes the space character.)

In [4]:
tokenizer.save_model('models/kantaiBERT')

['models/kantaiBERT\\vocab.json', 'models/kantaiBERT\\merges.txt']

Below include some samples tokenized using the above tokenizer.

In [6]:
tokenizer.encode('My name is Dilan and I am old').tokens

['My', 'Ġname', 'Ġis', 'ĠD', 'ilan', 'Ġand', 'ĠI', 'Ġam', 'Ġold']

But this is not enough for BERT processing. We need to make sure that sentences get seperated with related tokens (\<\s>, \<s>) as well. We can do this manually or we can use the transformer libnary provided BERTProcessing class to do that.

In [7]:
from tokenizers.processors import BertProcessing


tokenizer._tokenizer.post_processor = BertProcessing(
                            ("</s>", tokenizer.token_to_id("</s>")),
                            ("<s>", tokenizer.token_to_id("<s>"))
                        )

Now if we tokenize the same sentence as before, output would be like this.

In [8]:
tokenizer.encode('My name is Dilan and I am old').tokens

['<s>',
 'My',
 'Ġname',
 'Ġis',
 'ĠD',
 'ilan',
 'Ġand',
 'ĠI',
 'Ġam',
 'Ġold',
 '</s>']

Now tokenizer is in hand, we can move on to model configuration.

#### 3. Model Configuration

As we said earlier we will be training a RoBERTa type transformer with same number of layers and heads as DistilBERT model. So we will define our configurations accordingly.

In [10]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52000,
    max_postion_embedding=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

We will go into more details about these little later. 

But as you remember we earlier trained a tokenizer with base BytePairlevel class. Instead we will load the saved data of it using the RobertaTokenizer class.

In [8]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('models/kantaiBERT/', max_length=512)
tokenizer.convert_ids_to_tokens(tokenizer.encode("My name is Dilan and I am old."))

['<s>',
 'My',
 'Ġname',
 'Ġis',
 'ĠD',
 'ilan',
 'Ġand',
 'ĠI',
 'Ġam',
 'Ġold',
 '.',
 '</s>']

Now lets load the Roberta Model specified for the MLM task.

In [15]:
from transformers import RobertaForMaskedLM

# Initializing the model with previously created config data.
model = RobertaForMaskedLM(config=config)

In [16]:
print(model)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

#### 4. Prepare the Dataset

Now we need to prepare our dataset for training. We already have the tokenizer ready. So all we need to do is read the dataset and encode them in a proper manner.

In [ ]:
from dataclasses import dataclass
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(tokenizer, 'data/kant.txt', block_size=128)

Above code returns a dataset with encoded ids like below in Tensor format.

In [11]:
dataset.examples[1]

{'input_ids': tensor([   0, 1536, 2574,  300,  348,  267,  787,  270, 3525, 4460,  435,  512,
         3716,  305,  359,    2])}

In [12]:
tokenizer.convert_ids_to_tokens(dataset.examples[1]['input_ids'])

['<s>',
 'This',
 'ĠeBook',
 'Ġis',
 'Ġfor',
 'Ġthe',
 'Ġuse',
 'Ġof',
 'Ġanyone',
 'Ġanywhere',
 'Ġat',
 'Ġno',
 'Ġcost',
 'Ġand',
 'Ġwith',
 '</s>']

But we need to get this data as a batch for the training process. To do that we can either manually type the code or use the transformer provided DataCollator class like below.

In [24]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


Note the paremeters mlm and mlm_probability. These define the DataCollator to use MLM like data preparation and sets the token masking probability to 15%.

With those details Now we can move onto the actual model training.

In [25]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='models/kantaiBERT/',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_eval_batch_size=64,
    save_steps=10000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [26]:
trainer.train()

c:\Users\ddsdi\.virtualenvs\Pytorch_-_Use_Case_Exploration-FSDBtUdB\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 170964
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 21371
  2%|▏         | 503/21371 [00:37<22:16, 15.61it/s] 

{'loss': 7.3093, 'learning_rate': 4.8830190444995554e-05, 'epoch': 0.02}


  5%|▍         | 1003/21371 [01:09<22:10, 15.31it/s]

{'loss': 6.4482, 'learning_rate': 4.766038088999111e-05, 'epoch': 0.05}


  7%|▋         | 1503/21371 [01:42<21:59, 15.05it/s]

...

 98%|█████████▊| 21003/21371 [22:46<00:23, 15.64it/s]

{'loss': 4.075, 'learning_rate': 8.679986898132984e-07, 'epoch': 0.98}


100%|██████████| 21371/21371 [23:09<00:00, 16.26it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 21371/21371 [23:09<00:00, 15.38it/s]

{'train_runtime': 1389.9396, 'train_samples_per_second': 123.001, 'train_steps_per_second': 15.375, 'train_loss': 4.833053845191425, 'epoch': 1.0}


TrainOutput(global_step=21371, training_loss=4.833053845191425, metrics={'train_runtime': 1389.9396, 'train_samples_per_second': 123.001, 'train_steps_per_second': 15.375, 'train_loss': 4.833053845191425, 'epoch': 1.0})

Once the training is completed, we can save the model to the disk.

In [27]:
trainer.save_model('models/kantaiBERT/')

Saving model checkpoint to models/kantaiBERT/
Configuration saved in models/kantaiBERT/config.json
Model weights saved in models/kantaiBERT/pytorch_model.bin


#### 5. Evaluating the model



Now we can use our trained model for some basic inferencing tasks. For that we will use the trained model using transformer pipeline.

In [1]:
from cmath import pi
from transformers import pipeline

fill_mask = pipeline('fill-mask', 
                     model='models/kantaiBERT/', 
                     tokenizer='models/kantaiBERT/')

Once we get the relevant pipeline we can use it for our usecase like below.

In [7]:
fill_mask("Our thinking is based on our <mask>.")

[{'score': 0.021723872050642967,
  'token': 605,
  'token_str': ' conceptions',
  'sequence': 'Our thinking is based on our conceptions.'},
 {'score': 0.0203399658203125,
  'token': 418,
  'token_str': ' conception',
  'sequence': 'Our thinking is based on our conception.'},
 {'score': 0.018971821293234825,
  'token': 670,
  'token_str': ' principles',
  'sequence': 'Our thinking is based on our principles.'},
 {'score': 0.013673394918441772,
  'token': 600,
  'token_str': ' understanding',
  'sequence': 'Our thinking is based on our understanding.'},
 {'score': 0.013549803756177425,
  'token': 604,
  'token_str': ' existence',
  'sequence': 'Our thinking is based on our existence.'}]

The output is really interesting as you can see. It should be noted that this is trained only on very small set of data compared to what original transformers were trained. But still it yields a great result.

RoBERTa models are intersting as they provide more improved method for pretraining a BERT model. Read the techniques!